In [1]:
# test

import tensorflow as tf

hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

/Users/unahsu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


b'Hello, TensorFlow!'


In [2]:
import tensorflow as tf

sess = tf.InteractiveSession()

### Constants  
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

In [3]:
# constant of 1d tensor (vector)
a = tf.constant([2, 2], dtype=tf.int32, name="vector")
a.eval()

array([2, 2], dtype=int32)

In [4]:
# constant of 2x2 tensor (matrix)
b = tf.constant([[0, 1], [2, 3]], name="b")
b.eval()

array([[0, 1],
       [2, 3]], dtype=int32)

In [6]:
# We can also create tensors of a specific value.

c = tf.zeros([2, 3], tf.int32) # [[0, 0, 0], [0, 0, 0]]
print (c.eval())

d = tf.ones([2, 3], tf.int32) #  [[1, 1, 1], [1, 1, 1]]
print (d.eval())

[[0 0 0]
 [0 0 0]]
[[1 1 1]
 [1 1 1]]


In [7]:
# create a tensor containing zeros, with shape and type as input_tensor

input_tensor = tf.constant([[1,1], [2,2], [3,3]], dtype=tf.float32)
e = tf.zeros_like(input_tensor)  #  [[0, 0], [0, 0], [0, 0]]
print (e.eval())

f = tf.ones_like(input_tensor) # [[1, 1], [1, 1], [1, 1]]
print (f.eval())

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]]


### Variables  
Unlike a constant, a variable can be assigned to, so its value can be changed.   
Also, a constant's value is stored on the graph, whereas a variable's value is stored seperately.  
To declare a variable, we create a instance of tf.Variable.

In [8]:
#create variable a with scalar value
a = tf.Variable(2, name="scalar")

#create variable b as a vector
b = tf.Variable([2, 3], name="vector")

#create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")

# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

To assign value to variables, we can use tf.Variable.assign().  
It creates a operation that assigns the variable with the specified value.  
Also, it is important to remember that a variable needs to be initialized before used.  
To initialize variables, run tf.global_variables_initializer().

In [10]:
# assign a * 2 to a and call that op a_times_two
'''
# If a variable is used before initialized, an error will occur
a = tf.Variable(2, name="scalar")
a.eval() # a is NOT initialized

FailedPreconditionError
'''
a = tf.Variable(2, name="scalar")
a_times_two = a.assign(a*2) # an operation that assigns value a*2 to a

init = tf.global_variables_initializer() # an operation that initializes all variables
sess.run(init) # run the init operation with session
sess.run(a_times_two)

4

### Building a data flow graph  
A TensorFlow constant is a type of node which takes no inputs and outputs the value it stores. 

In [17]:
'''
node1 = tf.Variable(3.0, dtype=tf.float32)
node2 = tf.Variable(4.0) # also tf.float32 implicitly
node3 = tf.add(node1, node2) 

FailedPreconditionError
'''
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
node3 = tf.add(node1, node2) 

print(node1) 
print(node2)
print(node3)

Tensor("Const_5:0", shape=(), dtype=float32)
Tensor("Const_6:0", shape=(), dtype=float32)
Tensor("Add_3:0", shape=(), dtype=float32)


#### Visualizing and running a graph

In [18]:
# create a directory to store our graph
import os

logs_dir = './graph'
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

In [19]:
# To evaluate a graph, a Session is used. 
# A TensorFlow session places operations onto devices such as CPUs and GPUs and runs them, and computes variable values.

sess = tf.Session()
print(sess.run([node1, node2]))
print(sess.run(node3))
sess.close() # close the session

[3.0, 4.0]
7.0


In [20]:
with tf.Session() as sess:
    # write operations to the event file
    writer = tf.summary.FileWriter(logs_dir, sess.graph) 
    print(sess.run([node1, node2]))
    print(sess.run(node3))
    # no need to write sess.close()

writer.close() 

[3.0, 4.0]
7.0


### Placeholders and feed_dict  
Creating a graph of constants as the above is not particularly useful.  
A graph can be defined to accept external inputs without knowing the actual values needed for computation.  
  
A placeholder is used as a promise to provide a value later.  
Then, values are fed into the placeholder by providing a dictionary containing concrete values as argument for feed_dict.

In [22]:
'''
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5, 5, 5], tf.float32)
c = a + b # Short for tf.add(a, b)
with tf.Session() as sess:
    print(sess.run(c))

InvalidArgumentError
'''
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type float 32-bit, shape is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b # Short for tf.add(a, b)

with tf.Session() as sess:
    # feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
    # fetch value of c
    print(sess.run(c, feed_dict={a: [1, 2, 3]}))

[ 6.  7.  8.]


### Sharing Variables  
To share variables, we can explicitly pass tf.Variable objects or implicitly wrapping tf.Variable objects with tf.variable_scope objects.  
Variable scopes not only allow us to share variables, they also make naming variables easier.  
  
Suppose we have multi-layered model, instead of coming up with different names for variables in different layers.  
We can use different scopes to distinguish them. We can use tf.get_variable to get an existing variable, if the variable does not exist, a new one is created and returned.

In [23]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
    w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v")  # The same as v above.

In [24]:
# clear used variables in jupyter notebook
%reset -fs 